In [12]:
import numpy as np
import os, time, logging, traceback
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import tensorflow as tf
import tensorlayer as tl
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPooling2D
from keras.layers.merge import concatenate, add
from keras.utils import Sequence
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from error_handling_utilities import ModelFrameError
from error_handling_utilities import logClass as log
from DataGenerator import DataGenerator
from data_utilities import VisualizeImage, VisualizeImageWithPrediction
from model_utilities import LoadModelJSON, SaveModelJSON
from loss_utilities import dice_coef_loss, dice_coef, jaccard_distance_loss

In [13]:
class UNet:

    # NOTE: Private functions
    def __init__(self, shape:tuple, n_out = 1, dropout = 0.5, batchNorm = False, modelPath = None, modelName = None, loadModel = False):
        '''
        initialize the UNet model
        
        Arguments:
            shape {tuple of integer} -- the shape describing the input shape
        
        Keyword Arguments:
            n_out {int} -- the output size (default: {1})
            dropout {float} -- dropout prob. (default: {0.5})
            batchNorm {bool} -- status check to do batch normalization (default: {False})            
            modelPath {str} -- path of the saved model to load (default: {None})
            modelName {str} -- the name of the model to load (default: {None})
            loadModel {bool} -- status check for loading model(default: {False})
        '''
        # Setup model
        self.model = None
        if loadModel == True:
            self.model = self.LoadModel(path = modelPath, modelName = modelName)
            print("[INFO] Model loaded. Compile it")
            log.logger.info("[INFO] Model Loaded. Compile it!")

        else:
            self.model = self.__InitializeModel__(shape = shape, n_out = n_out, dropout = dropout, batchNorm = batchNorm)
            print("[INFO] Model built. Compile it!")
            log.logger.info("[INFO] Model built. Compile it!")

        # Setup folder list
        folderList = ["./checkpoint", 
        "./sample",
        "./test",
        "./savedModels"]
        self.__InitializeFolders__(folderList)
    
    def __InitializeFolders__(self, folderList: list) -> bool:
        '''
        Function to initialize all the required folders for the model to
        run. NOTE: PRIVATE FUNCTION -- DON'T USE
        
        Arguments:
            folderList {list} -- Consist of all the folders to initialize
        
        Returns:
            bool -- status return
        '''
        for folder in folderList:
            try:
                if not os.path.exists(folder):
                    os.mkdir(folder)
            except Exception as err:
                print("[FATAL] Exception occured while making folders: ", err)
                traceback.print_exc()
                log.logger.critical("[FATAL] Exception occured while making folders: " + err)
                return False
        print("[INFO] Folders initialized")
        log.logger.info("[INFO] Folders initialized")
        return True

    def __Conv2DBlock__(self, inputTensor, filters:int, kernelSize = 3, batchNorm = False):
        '''
        A conv2D block that defines a full 2 layer conv2d with batchnorm 
        # NOTE: PRIVATE FUNCTION -- DON'T USE

        Arguments:
            inputTensor {tensor} -- the input tensor
            filters {int} -- the total no. of filters

        Keyword Arguments:
            kernelSize {int} -- the size of each filter (default: {3})
            batchNorm {bool} -- status check for batchnorm (default: {False})
        '''
        # First layer
        x = Conv2D(filters = filters, kernel_size = (kernelSize, kernelSize), kernel_initializer = "he_normal", padding = "same")(inputTensor)
        if batchNorm:
            x = BatchNormalization()(x)
        x = Activation("relu")(x)

        # Second layer
        x = Conv2D(filters = filters, kernel_size = (kernelSize, kernelSize), kernel_initializer = "he_normal", padding = "same")(x)
        if batchNorm:
            x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x

    def __InitializeModel__(self, shape, n_out = 1, dropout = 0.5, batchNorm = False, ) -> Model:
        '''
        Function to initialize the UNet code # NOTE: PRIVATE FUNCTION -- DON'T USE
        
        Arguments:
            shape {tuple of int} -- shape of the input
        
        Keyword Arguments:
            n_out {int} -- the output size (default: {1})
            dropout {float} -- dropout prob. (default: {0.5})
            batchNorm {bool} -- status check to do batch normalization (default: {False})
        
        Returns:
            Model -- the builded model -- YET TO COMPILE
        '''

        nnx = int(shape[1])
        nny = int(shape[2])
        nnz = int(shape[3])
        print(" * Input: size of image: %d %d %d" % (nnx, nny, nnz))

        # Encoder 
        inputImage = Input((nnx, nny, nnz))
        # level 1
        conv1 = self.__Conv2DBlock__(inputTensor = inputImage, filters = 64, kernelSize = 3, batchNorm = batchNorm)
        pool1 = MaxPooling2D(pool_size = (2, 2))(conv1)
        pool1 = Dropout(dropout * 0.5)(pool1)
        # level 2
        conv2 = self.__Conv2DBlock__(inputTensor = pool1, filters = 128, kernelSize = 3, batchNorm = batchNorm)
        pool2 = MaxPooling2D(pool_size = (2, 2))(conv2)
        pool2 = Dropout(dropout)(pool2)
        # level 3
        conv3 = self.__Conv2DBlock__(inputTensor = pool2, filters = 256, kernelSize = 3, batchNorm = batchNorm)
        pool3 = MaxPooling2D(pool_size = (2, 2))(conv3)
        pool3 = Dropout(dropout)(pool3)
        # level 4
        conv4 = self.__Conv2DBlock__(inputTensor = pool3, filters = 512, kernelSize = 3, batchNorm = batchNorm)
        pool4 = MaxPooling2D(pool_size = (2, 2))(conv4)
        pool4 = Dropout(dropout)(pool4)
        # feature level -- last level
        conv5 = self.__Conv2DBlock__(inputTensor = pool4, filters = 1024, kernelSize = 3, batchNorm = batchNorm)
        
        # Decoder
        # level 4
        up1 = Conv2DTranspose(filters = 512, kernel_size = (3, 3), strides = (2, 2), padding = "same")(conv5)
        up1 = concatenate([up1, conv4])
        up1 = Dropout(dropout)(up1)
        conv6 = self.__Conv2DBlock__(inputTensor = up1, filters = 512, kernelSize = 3, batchNorm = batchNorm)
        # level 3
        up2 = Conv2DTranspose(filters = 256, kernel_size = (3, 3), strides = (2, 2), padding = "same")(conv6)
        up2 = concatenate([up2, conv3])
        up2 = Dropout(dropout)(up2)
        conv7 = self.__Conv2DBlock__(inputTensor = up2, filters = 256, kernelSize = 3, batchNorm = batchNorm)
        # level 2
        up3 = Conv2DTranspose(filters = 128, kernel_size = (3, 3), strides = (2, 2), padding = "same")(conv7)
        up3 = concatenate([up3, conv2])
        up3 = Dropout(dropout)(up3)
        conv8 = self.__Conv2DBlock__(inputTensor = up3, filters = 128, kernelSize = 3, batchNorm = batchNorm)
        # level 1
        up4 = Conv2DTranspose(filters = 64, kernel_size = (3, 3), strides = (2, 2), padding = "same")(conv8)
        up4 = concatenate([up4, conv1])
        up4 = Dropout(dropout)(up4)
        conv9 = self.__Conv2DBlock__(inputTensor = up4, filters = 64, kernelSize = 3, batchNorm = batchNorm)

        # output
        outputs = Conv2D(filters = n_out, kernel_size = (1, 1), activation = 'sigmoid')(conv9)
        
        # Make model
        model = Model(inputs = [inputImage], outputs = [outputs])

        return model


    # NOTE: Exposed API for usage
    def SetHyperParameters(self, learning_rate = 0.001, loss = "binary_crossentropy", metrics = ["accuracy"], batchSize = 5):
        '''
        Function to set the hyperparameters
        
        Keyword Arguments:
            learning_rate {float} -- rate at which training occurs (default: {0.001})
            loss {str} -- loss function (default: {"binary_crossentropy"})
            metrics {list} -- metrics to eval (default: {["accuracy"]})
            batchSize {int} -- size of the batch (default: {5})
        '''
        self.learningRate = learning_rate
        self.loss = loss
        self.metrics = metrics
        self.batchSize = batchSize
        print("[INFO] Hyperparams set")
        return True
    
    def SetupCallbacks(self, checkpoint_dir = "./checkpoint", verbose = 1):
        '''
        Function to setup callbacks
        
        Keyword Arguments:
            checkpoint_dir {str} -- [description] (default: {"./checkpoint"})
            verbose {int} -- [description] (default: {1})
        '''
        try:
            if not os.path.exists(checkpoint_dir):
                raise FileNotFoundError
            elif os.listdir(checkpoint_dir + "/") != []:
                raise FileExistsError
            else:
                True
        except FileNotFoundError:
            print("[WARN] Warning: Folder ./checkpoint/ don't exist hence making it")
            log.logger.warning("[WARN] Warning: Folder ./checkpoint/ don't exist hence making it")
            os.mkdir(checkpoint_dir)
        finally:
            self.callbacks = [
                EarlyStopping(patience = 10, verbose = verbose),
                ReduceLROnPlateau(factor = 0.1, patience = 3, min_lr = 0.000001, verbose = verbose),
                ModelCheckpoint(filepath = checkpoint_dir + "/checkpoint_model.h5", verbose = verbose, save_best_only = True, save_weights_only = True)
            ]
            print("[INFO] Callbacks set")
            return True
    
    def CompileModel(self, optimizer = "Adam") -> bool:
        '''
        Function to compile the built model
        
        Keyword Arguments:
            optimizer {str} -- the optimizer for the compilation (default: {"Adam"})
        
        Returns:
            bool -- Status check
        '''
        try:
            if not self.model:
                raise ModelFrameError
            if optimizer == "Adam":
                self.model.compile(
                    optimizer = Adam(lr = self.learningRate),
                    loss = self.loss,
                    metrics = self.metrics
                )
            else:
                self.model.compile(
                    optimizer = optimizer,
                    loss = self.loss,
                    metrics = self.metrics
                )
        except ModelFrameError:
            print("[WARN]: Build Model first!")
            log.logger.warning("[WARN]: Build Model first!")
            return False
        
        except Exception as err:
            print("[FATAL]: Compilation process failed with unknown exception: ", err)
            traceback.print_exc()
            log.logger.critical("[FATAL]: Compilation process failed with unknown exception: " + err)
            return False

        finally:
            return True
        
    def Train(self, XTrain, yTrain, XDev = None, yDev = None, n_epoch = 120):
        '''
        Function to train the data
        
        Arguments:
            XTrain {numpy array} -- input images for train 240 x 240 x 4
            yTrain {numpy array} -- label output of the image for train 240 x 240 x 1
        
        Keyword Arguments:
            XDev {[type]} -- input images for the validation 240 x 240 x 4 (default: {None})
            yDev {[type]} -- label output of the image for validation 240 x 240 x 1 (default: {None})
            n_epoch {int} -- the total number of the epoch (default: {120})
        '''
        # Set the data
        trainGen = DataGenerator(XSet = XTrain, ySet = yTrain, batch_size = self.batchSize)
        if XDev is not None and yDev is not None:
            devGen = DataGenerator(XSet = XDev, ySet = yDev, batch_size = self.batchSize)
        print("[INFO] Training started")
        log.logger.info("[INFO] Training started")

        # Train the model
        results = self.model.fit_generator(
            trainGen,
            steps_per_epoch = XTrain.shape[0] // self.batchSize,
            epochs = n_epoch,
            use_multiprocessing = True,
            shuffle = True,
            validation_data = devGen,
            validation_steps = XDev.shape[0] // self.batchSize,
            workers = 0)
        log.logger.info("[INFO] Train completed")
        # Plot and viz
        self.Plot(results = results)
           
    def Evaluate(self, XTest, yTest):
        '''
        Function to evaluate the model with test data
        
        Arguments:
            XTest {numpy array} -- X test data for testing
            yTest {numpy array} -- y label test data for testing
        '''

        testGen = DataGenerator(XTest, yTest, self.batchSize)
        self.model.evaluate_generator(
            testGen,
            steps = self.batchSize,
            verbose = 1,
            use_multiprocessing = True,
            workers = 0)
    
    def Predict(self, XTest, yTest):
        for ind, (x, y) in enumerate(zip(XTest, yTest)):
            pred = self.model.predict(x)
            VisualizeImageWithPrediction(x, y, pred, path = "./test/test_{}.png".format(ind))
        return True

    def Plot(self, results):
        plt.figure(figsize = (8, 8))
        plt.title("Learning Curve")
        plt.plot(results.history["loss"], label = "loss")
        plt.plot(results.history["val_loss"], label = "val_loss")
        plt.plot(np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker = "x", color = "r", label = "best model")
        plt.xlabel("Epochs")
        plt.ylabel("log_loss")
        plt.legend()

    def SaveModel(self, path = "./savedModels/", modelName = "UNet_Standard"):
        '''
        Function to save the model in the given directory
        
        Keyword Arguments:
            path {str} -- path to save the model (default: {"./savedModels/"})
            modelName {str} -- the name of the model (default: {"UNet_Standard"})
        '''
        log.logger.info("[INFO] Saving model as JSON and h5")
        SaveModelJSON(self.model, modelName, path)
    
    def LoadModelWeights(self, path = "./checkpoint/checkpoint_model.h5"):
        '''
        Function to load the weights alone given in .h5 file in the path

        Keyword Arguments:
            path {str} -- path for the weights (default: {"./checkpoint/checkpoint_model.h5"})
        '''
        self.model.load_weights(path)
        print("[INFO] Weight loaded")
        log.logger.info("[INFO] Weight loaded")
        return True
    
    def LoadModel(self, path = "./savedModels/", modelName = "UNet_Standard"):
        '''
        Function to load the model
        
        Keyword Arguments:
            path {str} -- the path for the saved model (default: {"./savedModels/"})
            modelName {str} -- the name of the model (default: {"UNet_Standard"})
        '''

        self.model = LoadModelJSON(modelName, path)
        print("[INFO] Model Loaded")
        log.logger.info("[INFO] Model loaded")
        return True


In [4]:
import dataset

28
28
28
(1, 240, 240, 153)
(1, 240, 240, 153)
(1, 240, 240, 153)
(1, 240, 240, 153)
{'DWI': {'mean': 23.145506, 'std': 46.76815}, 'Flair': {'mean': 28.769176, 'std': 57.13965}, 'T1': {'mean': 32.706516, 'std': 61.723255}, 'T2': {'mean': 51.48904, 'std': 101.01554}}
 HGG Validation
finished 22
finished 23
finished 24
 HGG Train
finished 1
finished 2
finished 3
finished 4
finished 5
finished 6
finished 7
finished 8
finished 9
finished 10
finished 11
finished 12
finished 13
finished 14
finished 15
finished 16
finished 17
finished 18
finished 19
finished 20
finished 21
(3213, 240, 240, 4)
(3213, 240, 240)
Preparing Testing Data
finished 25
finished 26
finished 27
finished 28
(612, 240, 240, 4)
(612, 240, 240)


In [5]:
XTrain = dataset.X_train_input
yTrain = dataset.X_train_target[:, :, :, np.newaxis]
XDev = dataset.X_dev_input
yDev = dataset.X_dev_target[:, :, :, np.newaxis]
XTest = dataset.X_test_input
yTest = dataset.X_test_target[:, :, :, np.newaxis]
print("[INFO] Data Loaded")

[INFO] Data Loaded


In [6]:
# Setup Data
yTrain = (yTrain > 0).astype(int)
yDev = (yDev > 0).astype(int)
yTest = (yTest > 0).astype(int)
X = np.asarray(XTrain[80])
y = np.asarray(yTrain[80])
nw, nh, nz = X.shape

In [7]:
print(XTrain.shape, yTrain.shape, XDev.shape, yDev.shape, XTest.shape, yTest.shape, sep = "\n")

(3213, 240, 240, 4)
(3213, 240, 240, 1)
(459, 240, 240, 4)
(459, 240, 240, 1)
(612, 240, 240, 4)
(612, 240, 240, 1)


In [8]:
# Hyperparameter settings
batch_size = 5
lr = 0.00001
n_epoch = 2

In [10]:
# Initialize model
model = UNet(shape = (batch_size, nw, nh, nz), dropout = 0.05, batchNorm = True)
model.SetHyperParameters(learning_rate = lr, batchSize = batch_size, loss = dice_coef_loss, metrics = [dice_coef])
model.SetupCallbacks()
model.CompileModel()

 * Input: size of image: 240 240 4
[INFO] Model built. Compile it!
[INFO] Folders initialized
[INFO] Hyperparams set
[INFO] Callbacks set


True

In [11]:
# Train model
model.Train(XTrain, yTrain, XDev, yDev, n_epoch = n_epoch)

[INFO] Training started
Epoch 1/2


177/642 [=======>......................] - ETA: 2:48:13 - loss: 0.2463 - dice_coef: 0.75 - ETA: 1:28:33 - loss: 0.2365 - dice_coef: 0.76 - ETA: 1:01:59 - loss: 0.2354 - dice_coef: 0.76 - ETA: 48:40 - loss: 0.2348 - dice_coef: 0.7652 - ETA: 40:41 - loss: 0.2347 - dice_coef: 0.76 - ETA: 35:20 - loss: 0.2353 - dice_coef: 0.76 - ETA: 31:32 - loss: 0.2359 - dice_coef: 0.76 - ETA: 28:41 - loss: 0.2368 - dice_coef: 0.76 - ETA: 26:27 - loss: 0.2373 - dice_coef: 0.76 - ETA: 24:40 - loss: 0.2378 - dice_coef: 0.76 - ETA: 23:14 - loss: 0.2378 - dice_coef: 0.76 - ETA: 22:01 - loss: 0.2377 - dice_coef: 0.76 - ETA: 20:58 - loss: 0.2376 - dice_coef: 0.76 - ETA: 20:05 - loss: 0.2372 - dice_coef: 0.76 - ETA: 19:19 - loss: 0.2366 - dice_coef: 0.76 - ETA: 18:38 - loss: 0.2358 - dice_coef: 0.76 - ETA: 18:02 - loss: 0.2350 - dice_coef: 0.76 - ETA: 17:30 - loss: 0.2340 - dice_coef: 0.76 - ETA: 17:01 - loss: 0.2331 - dice_coef: 0.76 - ETA: 16:35 - loss: 0.2321 - dice_coef: 0.76 - ETA: 16:11 - loss: 0.2313 - d

355/642 [===============>..............] - ETA: 7:21 - loss: 0.1743 - dice_coef: 0.825 - ETA: 7:20 - loss: 0.1741 - dice_coef: 0.825 - ETA: 7:19 - loss: 0.1739 - dice_coef: 0.826 - ETA: 7:18 - loss: 0.1738 - dice_coef: 0.826 - ETA: 7:16 - loss: 0.1736 - dice_coef: 0.826 - ETA: 7:15 - loss: 0.1733 - dice_coef: 0.826 - ETA: 7:14 - loss: 0.1731 - dice_coef: 0.826 - ETA: 7:13 - loss: 0.1728 - dice_coef: 0.827 - ETA: 7:12 - loss: 0.1726 - dice_coef: 0.827 - ETA: 7:11 - loss: 0.1725 - dice_coef: 0.827 - ETA: 7:10 - loss: 0.1723 - dice_coef: 0.827 - ETA: 7:08 - loss: 0.1722 - dice_coef: 0.827 - ETA: 7:07 - loss: 0.1720 - dice_coef: 0.828 - ETA: 7:06 - loss: 0.1717 - dice_coef: 0.828 - ETA: 7:05 - loss: 0.1714 - dice_coef: 0.828 - ETA: 7:04 - loss: 0.1712 - dice_coef: 0.828 - ETA: 7:03 - loss: 0.1708 - dice_coef: 0.829 - ETA: 7:02 - loss: 0.1705 - dice_coef: 0.829 - ETA: 7:01 - loss: 0.1703 - dice_coef: 0.829 - ETA: 7:00 - loss: 0.1700 - dice_coef: 0.830 - ETA: 6:58 - loss: 0.1696 - dice_coef:

533/642 [=======================>......] - ETA: 4:21 - loss: 0.1331 - dice_coef: 0.866 - ETA: 4:20 - loss: 0.1329 - dice_coef: 0.867 - ETA: 4:19 - loss: 0.1328 - dice_coef: 0.867 - ETA: 4:19 - loss: 0.1326 - dice_coef: 0.867 - ETA: 4:18 - loss: 0.1324 - dice_coef: 0.867 - ETA: 4:17 - loss: 0.1322 - dice_coef: 0.867 - ETA: 4:16 - loss: 0.1320 - dice_coef: 0.868 - ETA: 4:15 - loss: 0.1318 - dice_coef: 0.868 - ETA: 4:14 - loss: 0.1317 - dice_coef: 0.868 - ETA: 4:13 - loss: 0.1315 - dice_coef: 0.868 - ETA: 4:12 - loss: 0.1313 - dice_coef: 0.868 - ETA: 4:11 - loss: 0.1311 - dice_coef: 0.868 - ETA: 4:10 - loss: 0.1309 - dice_coef: 0.869 - ETA: 4:09 - loss: 0.1308 - dice_coef: 0.869 - ETA: 4:08 - loss: 0.1306 - dice_coef: 0.869 - ETA: 4:07 - loss: 0.1304 - dice_coef: 0.869 - ETA: 4:06 - loss: 0.1302 - dice_coef: 0.869 - ETA: 4:05 - loss: 0.1300 - dice_coef: 0.870 - ETA: 4:04 - loss: 0.1298 - dice_coef: 0.870 - ETA: 4:03 - loss: 0.1296 - dice_coef: 0.870 - ETA: 4:03 - loss: 0.1295 - dice_coef:

642/642 [==============================] - ETA: 1:37 - loss: 0.1051 - dice_coef: 0.894 - ETA: 1:36 - loss: 0.1050 - dice_coef: 0.895 - ETA: 1:35 - loss: 0.1049 - dice_coef: 0.895 - ETA: 1:34 - loss: 0.1047 - dice_coef: 0.895 - ETA: 1:34 - loss: 0.1046 - dice_coef: 0.895 - ETA: 1:33 - loss: 0.1045 - dice_coef: 0.895 - ETA: 1:32 - loss: 0.1044 - dice_coef: 0.895 - ETA: 1:31 - loss: 0.1042 - dice_coef: 0.895 - ETA: 1:30 - loss: 0.1041 - dice_coef: 0.895 - ETA: 1:29 - loss: 0.1040 - dice_coef: 0.896 - ETA: 1:28 - loss: 0.1039 - dice_coef: 0.896 - ETA: 1:27 - loss: 0.1037 - dice_coef: 0.896 - ETA: 1:26 - loss: 0.1036 - dice_coef: 0.896 - ETA: 1:25 - loss: 0.1035 - dice_coef: 0.896 - ETA: 1:24 - loss: 0.1034 - dice_coef: 0.896 - ETA: 1:24 - loss: 0.1033 - dice_coef: 0.896 - ETA: 1:23 - loss: 0.1032 - dice_coef: 0.896 - ETA: 1:22 - loss: 0.1031 - dice_coef: 0.896 - ETA: 1:21 - loss: 0.1030 - dice_coef: 0.897 - ETA: 1:20 - loss: 0.1028 - dice_coef: 0.897 - ETA: 1:19 - loss: 0.1027 - dice_coef:

ValueError: cannot reshape array of size 691200 into shape (5,240,240,4)

In [ ]:
# Eval model
model.Evaluate(XTest, yTest)

In [ ]:
# pred model
model.Predict(XTest, yTest)

In [ ]:
# Save model
model.SaveModel()

In [ ]:
# load model
model.LoadModel()
model.CompileModel()

In [ ]:
model.Evaluate(XTest, yTest)